In [1]:
!pip install selenium

  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-1.17.1-cp310-cp310-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.4 MB 7.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.4 MB 5.1 MB/s eta 0:00:02
   ----------- ---------------------------- 2.6/9.4 MB 5.4 MB/s eta 0:00:02
   ----------- ---------------------------- 2.6/9.4 MB 5.4 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.4 MB 2.8 MB/s eta 0:00:03
   ------------ --------------------------- 2.9/9.4 MB 2.8 MB/s eta 0:00:03
   ------------- --

In [3]:
!pip install beautifulsoup4

  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)

   ---------------------------------------- 0/2 [soupsieve]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   ---------------------------------------- 2/2 [beautifulsoup4]



In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os
import time
import csv

In [8]:
options = Options()
options.add_argument('--headless')  # Opsional: aktifkan/komentari sesuai kebutuhan
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--window-size=1920,1080")

chromedriver_path = "C:/Users/danny/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)


In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import csv

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--window-size=1920,1080")
options.add_argument("--user-data-dir=/tmp/profil_chrome_unik")

driver = webdriver.Chrome(options=options)

def get_recipe_links_sehat(max_pages=771):
    links = set()
    for page in range(1, max_pages + 1):
        url = f"https://cookpad.com/id/cari/clean%20food?page={page}"
        print(f"[PAGE {page}] {url}")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if href.startswith("/id/resep/") and href.count('-') > 1:
                full_url = "https://cookpad.com" + href.split("?")[0]
                links.add(full_url)
    return list(links)

def scrape_recipe(url):
    try:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        title = soup.find("h1").text.strip()
        ingredients = [item.text.strip() for item in soup.select(".ingredients-list li")]
        steps = [item.text.strip() for item in soup.select('[data-collapse-recipe-target="step"]')]

        return {
            "judul": title,
            "url": url,
            "bahan": ingredients,
            "langkah": steps
        }
    except Exception as e:
        print(f"Gagal scrape {url}: {e}")
        return None

def simpan_batch_csv(data_batch, filename, write_header=False):
    mode = "a" if os.path.exists(filename) else "w"
    with open(filename, mode, newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header and mode == "w":
            writer.writerow(["Judul", "URL", "Bahan", "Langkah"])
        for r in data_batch:
            writer.writerow([
                r["judul"],
                r["url"],
                "; ".join(r["bahan"]),
                " | ".join(r["langkah"])
            ])

max_pages = 200
batch_size = 50
output_file = "resep_clean_food.csv"

recipe_links = get_recipe_links_sehat(max_pages)
print(f"Ditemukan {len(recipe_links)} link resep")

all_recipes = []
for i, link in enumerate(recipe_links, 1):
    print(f"[{i}] Scraping: {link}")
    data = scrape_recipe(link)
    if data:
        all_recipes.append(data)

    if i % batch_size == 0 or i == len(recipe_links):
        print(f" Menyimpan batch ke-{i // batch_size + 1}")
        simpan_batch_csv(all_recipes, output_file, write_header=(i <= batch_size))
        all_recipes = []

    time.sleep(2)

driver.quit()
print(" Selesai! Data disimpan di:", output_file)


[PAGE 1] https://cookpad.com/id/cari/clean%20food?page=1
[PAGE 2] https://cookpad.com/id/cari/clean%20food?page=2
[PAGE 3] https://cookpad.com/id/cari/clean%20food?page=3
[PAGE 4] https://cookpad.com/id/cari/clean%20food?page=4
[PAGE 5] https://cookpad.com/id/cari/clean%20food?page=5
[PAGE 6] https://cookpad.com/id/cari/clean%20food?page=6
[PAGE 7] https://cookpad.com/id/cari/clean%20food?page=7
[PAGE 8] https://cookpad.com/id/cari/clean%20food?page=8
[PAGE 9] https://cookpad.com/id/cari/clean%20food?page=9
[PAGE 10] https://cookpad.com/id/cari/clean%20food?page=10
[PAGE 11] https://cookpad.com/id/cari/clean%20food?page=11
[PAGE 12] https://cookpad.com/id/cari/clean%20food?page=12
[PAGE 13] https://cookpad.com/id/cari/clean%20food?page=13
[PAGE 14] https://cookpad.com/id/cari/clean%20food?page=14
[PAGE 15] https://cookpad.com/id/cari/clean%20food?page=15
[PAGE 16] https://cookpad.com/id/cari/clean%20food?page=16
[PAGE 17] https://cookpad.com/id/cari/clean%20food?page=17
[PAGE 18] https